In [11]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

In [12]:
dataset = '../Dataset/Car/car'
#dataset = '../Dataset/Balance-sheet/train.txt'
#dataset = '../Dataset/Hayes-Roth/Hayes-Roth'
#dataset = '../Dataset/Heart-Data/train.txt'
#dataset = '../Dataset/TicTacToe/tic tac toe'
DATASET_SPLIT_RATIO = .1
np.random.seed(2)

In [13]:
class DecisionTreeClassifier:
    def __init__(self):
        self.number_of_nodes = 1
        self.root_node = 0
        self.node_identity = [-1]
        self.node_edge = [-1]
        self.node_edge_dict = [-1]
        self.mx_identity= [-1]

    def select_feature(self, data):
        list_of_features = list(data)
        list_of_labels = np.unique(data[list_of_features[-1]], return_counts=True)[0]
        label_count = np.unique(data[list_of_features[-1]], return_counts=True)[1]
        total_number_of_rows = len(data)
        entropy = 0
        for i in range(0, len(list_of_labels)):
            entropy = entropy + (- label_count[i]/total_number_of_rows) * (math.log2(label_count[i]/total_number_of_rows))
            # Total numer of rows in the entire dataset or the number of rows in the current node?
        #print('Entropy:', entropy)

        mx_gain_ratio = -1
        mx_idx = -1

        for i in range(0, len(list_of_features) - 1):
            # calculate information gain of i-th feature using the labels on data[list_of_features[-1]]
            list_of_possible_values = np.unique(data[list_of_features[i]], return_counts=True)[0]
            count_values = np.unique(data[list_of_features[i]], return_counts=True)[1]
            total_values = sum(count_values)
            information = 0
            split_info = 0
            #print(list_of_possible_values)
            #print(count_values)
            for j in range(len(list_of_possible_values)):
                information_label = 0
                for k in range(len(list_of_labels)):
                    denominator = len(data[(data[list_of_features[i]] == list_of_possible_values[j]) & (data[list(data)[-1]] == list_of_labels[k])])          
                    if denominator == 0:
                        continue
                    information_label = information_label + (- denominator / count_values[j]) * math.log2(denominator / count_values[j])
                information = information + information_label * (count_values[j]/total_values)
                split_info = split_info +  -(count_values[j]/total_values) * math.log2(count_values[j]/total_values)

            information_gain = entropy - information
            #split_info = 1
            gain_ratio = information_gain / split_info
            #print('Feature', i, ':', information_gain, split_info, gain_ratio)
            if gain_ratio > mx_gain_ratio:
                mx_gain_ratio = gain_ratio
                mx_idx = i

            #print('Feature', i, ':', information)
            #print('Feature', i, ':', information_gain)

        return mx_idx
    def train(self, node, data):
        list_of_features = list(data)
        list_of_labels = np.unique(data[list_of_features[-1]], return_counts=True)[0]
        list_of_labels_count = np.unique(data[list_of_features[-1]], return_counts=True)[1]
        if len(list_of_labels) == 1:
            self.node_identity[node] = list_of_labels[0]
            return
        majority_idx = np.argmax(list_of_labels_count)
        self.mx_identity[node] = list_of_labels[majority_idx]
        if len(list(data)) == 1:
            majority_idx = np.argmax(list_of_labels_count)
            self.node_identity[node] = list_of_labels[majority_idx] 
            return

        mx_idx = self.select_feature(data)
        #print(node, mx_idx)
        remaining_features = list_of_features[0:mx_idx] + list_of_features[mx_idx+1: ]
        #print('Selected feature:', mx_idx)
        #print('Remaining_features:', remaining_features)
        list_of_possible_values = np.unique(data[list_of_features[mx_idx]])
        self.node_edge[node] = list_of_features[mx_idx]
        self.node_edge_dict[node] = {}    
        for label in list_of_possible_values:
            new_data_frame = data[data[list_of_features[mx_idx]] == label][remaining_features]
            if len(new_data_frame) == 0:
                continue
            new_node = self.number_of_nodes
            self.node_edge_dict[node][label] = new_node
            self.node_identity.append(-1)
            self.node_edge.append(-1)
            self.node_edge_dict.append(-1)
            self.mx_identity.append(-1)
            self.number_of_nodes += 1
            self.train(new_node, new_data_frame)
        return 
    def predict(self, data):
        cur_node = 0
        while self.node_identity[cur_node] == -1:
            try:
                cur_node = self.node_edge_dict[cur_node][data[self.node_edge[cur_node]].values[0]]
            except:
                return self.mx_identity[cur_node]
        return self.node_identity[cur_node]
    def test(self, test):
        right_predictions = 0
        for i in range(len(test)):
            if self.predict(test[i:i+1]) == test[i:i+1][list(test)[-1]].values[0]:
                right_predictions+=1
        return round(right_predictions/len(test) * 100, 2)

In [14]:
def cross_validation(dataframe):
    decision_tree_classifier = DecisionTreeClassifier()
    accuracies = []
    data_in_a_single_fold = int(np.ceil(max(10, len(dataframe)/10)))
    #data_in_a_single_fold = int(np.ceil(.1*len(dataframe)))
    number_of_folds = int(np.ceil(len(dataframe)/data_in_a_single_fold))
    print('Number of folds:', number_of_folds)
    print('Number of rows in a single fold:', data_in_a_single_fold)
    for i in range(number_of_folds):
        print(min((i)*data_in_a_single_fold, len(dataframe)), 'bal')
        a = dataframe.iloc[0 : min((i)*data_in_a_single_fold, len(dataframe))]
        try:
            print('min', min((i+1) * data_in_a_single_fold, len(dataframe)))
            b = dataframe.iloc[min((i+1) * data_in_a_single_fold, len(dataframe)) :  len(dataframe)]
            print(b.head())
            train = pd.concat(a, b).reset_index().drop(columns = 'index')
            b.reset_index().drop(columns = 'index')
            print('haha', train.head())
        except:
            if len(a) == 0:
                train = b
            else:
                train = a
        
        test = dataframe.iloc[i * data_in_a_single_fold : min((i+1)*data_in_a_single_fold, len(dataframe))].reset_index().drop(columns = 'index')
        print('Number of training samples:', len(train))
        print('Number of test samples:', len(test))
        print('Labels:', np.unique(train[list(dataframe)[-1]]))
        decision_tree_classifier.train(0, train)    
        accuracies.append(decision_tree_classifier.test(test))
    return accuracies

In [15]:
dataframe = pd.read_csv(dataset, header=None)
dataframe = dataframe.sample(frac=1)
accuracies = cross_validation(dataframe)
print('Accuracies:', accuracies)
print('Average accuracy:', round(np.mean(accuracies), 2))

Number of folds: 10
Number of rows in a single fold: 173
0 bal
min 173
          0      1      2     3      4     5      6
1084    med    med      2     2    med   med  unacc
83    vhigh  vhigh  5more     2  small  high  unacc
1443    low   high      3     4    med   low  unacc
1310    low  vhigh      2     4    med  high    acc
1048    med   high      4  more    med   med    acc
Number of training samples: 1555
Number of test samples: 173
Labels: ['acc' 'good' 'unacc' 'vgood']
173 bal
min 346
         0     1  2     3    4     5      6
599   high  high  4     2  med  high  unacc
678   high   med  3     2  med   low  unacc
728   high   med  4  more  big  high    acc
1043   med  high  4     4  big  high    acc
787   high   low  3     2  med   med  unacc
Number of training samples: 173
Number of test samples: 173
Labels: ['acc' 'good' 'unacc' 'vgood']
346 bal
min 519
          0      1  2  3      4     5      6
568    high   high  3  2  small   med  unacc
70    vhigh  vhigh  4  4    big 

/home/immuntasir/.local/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars


519 bal
min 692
         0      1      2     3      4    5      6
523   high  vhigh  5more     4  small  med  unacc
721   high    med      4  more  small  med  unacc
1150   med    med      4     4    big  med    acc
1303   low  vhigh      2     2    big  med  unacc
682   high    med      3     2    big  med  unacc
Number of training samples: 519
Number of test samples: 173
Labels: ['acc' 'good' 'unacc' 'vgood']
692 bal
min 865
         0      1      2     3    4     5      6
897    med  vhigh      3     2  big   low  unacc
552   high   high      2     4  med   low  unacc
511   high  vhigh      4  more  big   med  unacc
745   high    med  5more     4  big   med    acc
1583   low    med      4     4  big  high  vgood
Number of training samples: 692
Number of test samples: 173
Labels: ['acc' 'good' 'unacc' 'vgood']
865 bal
min 1038
         0      1      2     3      4     5      6
971    med  vhigh  5more  more    big  high    acc
710   high    med      4     2    big  high  unacc
1062  

In [16]:
dataframe.iloc[0:0]

,0,1,2,3,4,5,6


In [7]:
#train, test = train_test_split(dataframe, test_size=DATASET_SPLIT_RATIO)
#print('Accuacy:', str(  round(right_predictions/len(test) * 100, 2)) + '%' )

In [8]:
print(len(dataframe))

1728


In [9]:
train.head()
print('Number of training samples:', len(train))
print('Number of test samples:', len(test))
print('Labels:', np.unique(train[list(dataframe)[-1]]))

NameError: name 'train' is not defined

Car Dataset: 93.06%
Balance Sheet: 26.98%
Hayes-Roth: 50.0%
Heart Data: 89.47%
TicTacToe: 85.42%